# Face Generation

In this project, you'll define and train a Generative Adverserial network of your own creation on a dataset of faces. Your goal is to get a generator network to generate *new* images of faces that look as realistic as possible!

The project will be broken down into a series of tasks from **defining new architectures training adversarial networks**. At the end of the notebook, you'll be able to visualize the results of your trained Generator to see how it performs; your generated samples should look like fairly realistic faces with small amounts of noise.

### Get the Data

You'll be using the [CelebFaces Attributes Dataset (CelebA)](http://mmlab.ie.cuhk.edu.hk/projects/CelebA.html) to train your adversarial networks.

This dataset has higher resolution images than datasets you have previously worked with (like MNIST or SVHN) you've been working with, and so, you should prepare to define deeper networks and train them for a longer time to get good results. It is suggested that you utilize a GPU for training.

### Pre-processed Data

Since the project's main focus is on building the GANs, we've done *some* of the pre-processing for you. Each of the CelebA images has been cropped to remove parts of the image that don't include a face, then resized down to 64x64x3 NumPy images. Some sample data is show below.

<img src='https://github.com/karencfisher/GAN-experiments/blob/main/dlnd_face_generation/assets/processed_face_data.png?raw=1' width=60% />

> If you are working locally, you can download this data [by clicking here](https://s3.amazonaws.com/video.udacity-data.com/topher/2018/November/5be7eb6f_processed-celeba-small/processed-celeba-small.zip)

This is a zip file that you'll need to extract in the home directory of this notebook for further loading and processing. After extracting the data, you should be left with a directory of data `processed-celeba-small/`.

In [ ]:
# This was to clone the repository and change directory to the project folder for training on Colab
!git clone https://github.com/karencfisher/GAN-experiments.git
%cd GAN-experiments/dlnd_face_generation

In [ ]:
# run this once to unzip the file
!wget https://s3.amazonaws.com/video.udacity-data.com/topher/2018/November/5be7eb6f_processed-celeba-small/processed-celeba-small.zip
!unzip -qq processed-celeba-small.zip


In [ ]:
from glob import glob
from typing import Tuple, Callable, Dict
import os

import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
from PIL import Image
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

import tests
from eigenloss import get_eigenloss

In [ ]:
data_dir = 'processed_celeba_small/celeba/'

## Data pipeline

The [CelebA](http://mmlab.ie.cuhk.edu.hk/projects/CelebA.html) dataset contains over 200,000 celebrity images with annotations. Since you're going to be generating faces, you won't need the annotations, you'll only need the images. Note that these are color images with [3 color channels (RGB)](https://en.wikipedia.org/wiki/Channel_(digital_image)#RGB_Images) each.

### Pre-process and Load the Data

Since the project's main focus is on building the GANs, we've done *some* of the pre-processing for you. Each of the CelebA images has been cropped to remove parts of the image that don't include a face, then resized down to 64x64x3 NumPy images. This *pre-processed* dataset is a smaller subset of the very large CelebA dataset and contains roughly 30,000 images.

Your first task consists in building the dataloader. To do so, you need to do the following:
* implement the get_transforms function
* create a custom Dataset class that reads the CelebA data

### Exercise: implement the get_transforms function

The `get_transforms` function should output a [`torchvision.transforms.Compose`](https://pytorch.org/vision/stable/generated/torchvision.transforms.Compose.html#torchvision.transforms.Compose) of different transformations. You have two constraints:
* the function takes a tuple of size as input and should **resize the images** to the input size
* the output images should have values **ranging from -1 to 1**

In [ ]:
def get_transforms(size: Tuple[int, int]) -> Callable:
    """ Transforms to apply to the image."""

    transform = transforms.Compose([
        transforms.Resize(size),
        transforms.CenterCrop(size),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    return transform

### Exercise: implement the DatasetDirectory class


The `DatasetDirectory` class is a torch Dataset that reads from the above data directory. The `__getitem__` method should output a transformed tensor and the `__len__` method should output the number of files in our dataset. You can look at [this custom dataset](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html#creating-a-custom-dataset-for-your-files) for ideas.

In [ ]:
class DatasetDirectory(Dataset):
    """
    A custom dataset class that loads images from folder.
    args:
    - directory: location of the images
    - transform: transform function to apply to the images
    - extension: file format
    """
    def __init__(self,
                 directory: str,
                 transforms: Callable = None,
                 extension: str = '.jpg'):
        self.directory = directory
        self.transforms = transforms
        self.extension = extension

        # List all files in the directory with the given extension
        self.files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith(extension)]

    def __len__(self) -> int:
        """ returns the number of items in the dataset """
        return len(self.files)

    def __getitem__(self, index: int) -> torch.Tensor:
        """ load an image and apply transformation """
        # Load the image at the given index
        img_path = self.files[index]
        image = Image.open(img_path).convert('RGB')

        # Apply transformations if provided
        if self.transforms:
            image = self.transforms(image)

        return image

In [ ]:
"""
DO NOT MODIFY ANYTHING IN THIS CELL
"""
# run this cell to verify your dataset implementation
dataset = DatasetDirectory(data_dir, get_transforms((64, 64)))
tests.check_dataset_outputs(dataset)

The functions below will help you visualize images from the dataset.

In [ ]:
"""
DO NOT MODIFY ANYTHING IN THIS CELL
"""

def denormalize(images):
    """Transform images from [-1.0, 1.0] to [0, 255] and cast them to uint8."""
    return ((images + 1.) / 2. * 255).astype(np.uint8)

# plot the images in the batch, along with the corresponding labels
fig = plt.figure(figsize=(20, 4))
plot_size=20
for idx in np.arange(plot_size):
    ax = fig.add_subplot(2, int(plot_size/2), idx+1, xticks=[], yticks=[])
    img = dataset[idx].numpy()
    img = np.transpose(img, (1, 2, 0))
    img = denormalize(img)
    ax.imshow(img)

## Model implementation

As you know, a GAN is comprised of two adversarial networks, a discriminator and a generator. Now that we have a working data pipeline, we need to implement the discriminator and the generator.

Feel free to implement any additional class or function.

### Exercise: Create the discriminator

The discriminator's job is to score real and fake images. You have two constraints here:
* the discriminator takes as input a **batch of 64x64x3 images**
* the output should be a single value (=score)

Feel free to get inspiration from the different architectures we talked about in the course, such as DCGAN, WGAN-GP or DRAGAN.

#### Some tips
* To scale down from the input image, you can either use `Conv2d` layers with the correct hyperparameters or Pooling layers.
* If you plan on using gradient penalty, do not use Batch Normalization layers in the discriminator.

In [ ]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, use_batch_norm=True):
        super(ConvBlock, self).__init__()
        self.use_batch_norm = use_batch_norm

        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride=2,
                              padding = 1, bias=False)
        self.batch_norm = nn.BatchNorm2d(out_channels)
        self.activation = nn.LeakyReLU(0.2)

    def forward(self, x):
        x = self.conv(x)
        x = self.batch_norm(x) if self.use_batch_norm else x
        return self.activation(x)


class Discriminator(nn.Module):
    def __init__(self, conv_dim=64, channels=3):  # Increased conv_dim
        super(Discriminator, self).__init__()
        self.discriminator = nn.Sequential(
            ConvBlock(channels, conv_dim, 4, use_batch_norm=False),
            ConvBlock(conv_dim, conv_dim * 2, 4),
            ConvBlock(conv_dim * 2, conv_dim * 4, 4),
            ConvBlock(conv_dim * 4, conv_dim * 8, 4),
            ConvBlock(conv_dim * 8, conv_dim * 16, 4),

            nn.Flatten(),
            nn.Linear(conv_dim * 16 * 2 * 2, 1)  # Adjusted for 64x64 output
        )

    def forward(self, x):
        out = self.discriminator(x)
        return out.view([-1, 1, 1, 1])

In [ ]:
"""
DO NOT MODIFY ANYTHING IN THIS CELL
"""
# run this cell to check your discriminator implementation
discriminator = Discriminator()
tests.check_discriminator(discriminator)

### Exercise: create the generator

The generator's job creates the "fake images" and learns the dataset distribution. You have three constraints here:
* the generator takes as input a vector of dimension `[batch_size, latent_dimension, 1, 1]`
* the generator must outputs **64x64x3 images**

Feel free to get inspiration from the different architectures we talked about in the course, such as DCGAN, WGAN-GP or DRAGAN.

#### Some tips:
* to scale up from the latent vector input, you can use `ConvTranspose2d` layers
* as often with Gan, **Batch Normalization** helps with training

In [ ]:
class DeconvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride,
                 padding, use_batch_norm=True):
        super(DeconvBlock, self).__init__()
        self.use_batch_norm = use_batch_norm
        self.deconv = nn.ConvTranspose2d(in_channels, out_channels, kernel_size,
                                         stride, padding, bias=False)
        self.batch_norm = nn.BatchNorm2d(out_channels)
        self.activation = nn.ReLU()

    def forward(self, x):
        x = self.deconv(x)
        x = self.batch_norm(x) if self.use_batch_norm else x
        return self.activation(x)

class Generator(nn.Module):
    def __init__(self, latent_dim, conv_dim=64, channels=3):
        super(Generator, self).__init__()
        self.latent_dim = latent_dim
        self.generator = nn.Sequential(
            DeconvBlock(latent_dim, conv_dim * 16, 4, 2, 1),
            DeconvBlock(conv_dim * 16, conv_dim * 8, 4, 2, 1),
            DeconvBlock(conv_dim * 8, conv_dim * 4, 4, 2, 1),
            DeconvBlock(conv_dim * 4, conv_dim * 2, 4, 2, 1),
            DeconvBlock(conv_dim * 2, conv_dim, 4, 2, 1),

            nn.ConvTranspose2d(conv_dim, channels, 4, stride=2, padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        return self.generator(x)

In [ ]:
"""
DO NOT MODIFY ANYTHING IN THIS CELL
"""
# run this cell to verify your generator implementation
latent_dim = 128
generator = Generator(latent_dim)
tests.check_generator(generator, latent_dim)

## Optimizer

In the following section, we create the optimizers for the generator and discriminator. You may want to experiment with different optimizers, learning rates and other hyperparameters as they tend to impact the output quality.

### Exercise: implement the optimizers

In [ ]:
import torch.optim as optim


def create_optimizers(generator: nn.Module, discriminator: nn.Module, lr):
    """ This function should return the optimizers of the generator and the discriminator """
    # TODO: implement the generator and discriminator optimizers
    g_optimizer = optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
    d_optimizer = optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))
    return g_optimizer, d_optimizer

## Losses implementation

In this section, we are going to implement the loss function for the generator and the discriminator. You can and should experiment with different loss function.

Some tips:
* You can choose the commonly used the binary cross entropy loss or select other losses we have discovered in the course, such as the Wasserstein distance.
* You may want to implement a gradient penalty function as discussed in the course. It is not required and the code will work whether you implement it or not.

### Exercise: implement the generator loss

The generator's goal is to get the discriminator to think its generated images (= "fake" images) are real.

In [ ]:
def generator_loss(fake_logits, device, smoothing=0.9):
    fake_logits = fake_logits.view(fake_logits.size(0), -1)
    labels = torch.ones_like(fake_logits) * smoothing
    labels = labels.to(device)
    criterion = nn.BCEWithLogitsLoss()
    return criterion(fake_logits, labels)

### Exercise: implement the discriminator loss

We want the discriminator to give high scores to real images and low scores to fake ones and the discriminator loss should reflect that.

In [ ]:
def discriminator_loss(real_logits, fake_logits, device, smoothing=0.9):
    real_logits = real_logits.view(real_logits.size(0), -1)
    fake_logits = fake_logits.view(fake_logits.size(0), -1)
    real_labels = (torch.ones_like(real_logits) * smoothing).to(device)
    fake_labels = torch.zeros_like(fake_logits).to(device)
    criterion = nn.BCEWithLogitsLoss()
    loss = criterion(real_logits, real_labels) + criterion(fake_logits, fake_labels)
    return loss

### Exercise (Optional): Implement the gradient Penalty

In the course, we discussed the importance of gradient penalty in training certain types of Gans. Implementing this function is not required and depends on some of the design decision you made (discriminator architecture, loss functions).

In [ ]:
def gradient_penalty(discriminator, real_samples, fake_samples):
    """ This function enforces """
    gp = 0
    # TODO (Optional): implement the gradient penalty
    return gp

## Training


Training will involve alternating between training the discriminator and the generator. You'll use your functions real_loss and fake_loss to help you calculate the discriminator losses.

* You should train the discriminator by alternating on real and fake images
* Then the generator, which tries to trick the discriminator and should have an opposing loss function

### Exercise: implement the generator step and the discriminator step functions

Each function should do the following:
* calculate the loss
* backpropagate the gradient
* perform one optimizer step

In [ ]:
def generator_step(generator: nn.Module, discriminator: nn.Module,
                   gen_optimizer: torch.optim.Optimizer, batch_size: int,
                   latent_dim: int, device: str, smoothing: float) -> Dict:
    """ One training step of the generator. """
    generator.train()
    gen_optimizer.zero_grad()

    noise = np.random.normal(0, 1, (batch_size, latent_dim, 1, 1))
    noise = torch.tensor(noise, dtype=torch.float).to(device)
    fake_images = generator(noise)
    fake_logits = discriminator(fake_images)

    g_loss = generator_loss(fake_logits, device, smoothing)
    g_loss.backward()
    gen_optimizer.step()

    return {'loss': g_loss.item()}

def discriminator_step(discriminator: nn.Module, generator: nn.Module,
                       disc_optimizer: torch.optim.Optimizer, batch_size: int,
                       latent_dim: int, real_images: torch.Tensor,
                       device: str, smoothing: float) -> Dict:
    """ One training step of the discriminator. """
    discriminator.train()
    disc_optimizer.zero_grad()

    # Generate fake images
    noise = np.random.normal(0, 1, (batch_size, latent_dim, 1, 1))
    noise = torch.tensor(noise, dtype=torch.float).to(device)
    fake_images = generator(noise)

    # Get discriminator logits for real and fake images
    real_logits = discriminator(real_images)
    fake_logits = discriminator(fake_images.detach())

    # Calculate losses
    d_loss = discriminator_loss(real_logits, fake_logits, device, smoothing)
    d_loss.backward()
    disc_optimizer.step()

    return {'loss': d_loss.item()}

### Main training loop

You don't have to implement anything here but you can experiment with different hyperparameters.

In [ ]:
from datetime import datetime

In [ ]:
# you can experiment with different dimensions of latent spaces
latent_dim = 128

# update to cpu if you do not have access to a gpu
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# number of epochs to train your model
n_epochs = 30

# number of images in each batch
batch_size = 64

#learning rate
lr = 0.0002

# Smoothing factor (1.0 for none)
smoothing = 0.9

In [ ]:
"""
DO NOT MODIFY ANYTHING IN THIS CELL
"""
print_every = 50

# Create optimizers for the discriminator D and generator G
generator = Generator(latent_dim).to(device)
discriminator = Discriminator().to(device)
g_optimizer, d_optimizer = create_optimizers(generator, discriminator, lr)

dataloader = DataLoader(dataset,
                        batch_size=64,
                        shuffle=True,
                        num_workers=4,
                        drop_last=True,
                        pin_memory=False)

In [ ]:
"""
DO NOT MODIFY ANYTHING IN THIS CELL
"""

def display(fixed_latent_vector: torch.Tensor):
    """ helper function to display images during training """
    fig = plt.figure(figsize=(14, 4))
    plot_size = 16
    for idx in np.arange(plot_size):
        ax = fig.add_subplot(2, int(plot_size/2), idx+1, xticks=[], yticks=[])
        img = fixed_latent_vector[idx, ...].detach().cpu().numpy()
        img = np.transpose(img, (1, 2, 0))
        img = denormalize(img)
        ax.imshow(img)
    plt.show()

### Exercise: implement the training strategy

You should experiment with different training strategies. For example:

* train the generator more often than the discriminator.
* added noise to the input image
* use label smoothing

Implement with your training strategy below.

In [ ]:
print('Training the eigenloss...')
gen_eigenloss = get_eigenloss(dataloader, n_components=200, n_batches=200)

In [ ]:
fixed_latent_vector = torch.randn(128, latent_dim, 1, 1).float().to(device)

losses = []
eigen_losses = []
minimum_eigenloss = 1.0
for epoch in range(n_epochs):
    for batch_i, real_images in enumerate(dataloader):
        real_images = real_images.to(device)

        # train the discriminator
        d_loss = discriminator_step(discriminator, generator, d_optimizer,
                                    batch_size, latent_dim, real_images,
                                    device, smoothing)

        # train the generator
        g_loss = generator_step(generator, discriminator, g_optimizer,
                                batch_size, latent_dim, device, smoothing)

        if batch_i % print_every == 0:
            # append discriminator loss and generator loss
            d = d_loss['loss']
            g = g_loss['loss']
            losses.append((d, g))
            # print discriminator and generator loss
            time = str(datetime.now()).split('.')[0]
            print(f'\r{time} | Epoch [{epoch+1}/{n_epochs}] | Batch {batch_i}/{len(dataloader)} | ',
                  f'd_loss: {d:.4f} | g_loss: {g:.4f}', end='')

    # display images during training
    generator.eval()
    generated_images = generator(fixed_latent_vector)
    print('\n')
    display(generated_images[:16])
    
    # compute eigenloss to evaluate generated images
    eigen_loss = gen_eigenloss.compute_loss(generated_images).item()
    eigen_losses.append(eigen_loss)
    print(f'Eigenloss of generated images: {eigen_loss}')
    if eigen_loss < minimum_eigenloss:
        print('New minimum, saving generator weights...')
        minimum_eigenloss = eigen_loss
        torch.save(generator.state_dict(), 'generator.pth')
    print('')
    
    

### Training losses

Plot the training losses for the generator and discriminator.

In [ ]:
"""
DO NOT MODIFY ANYTHING IN THIS CELL
"""
fig, ax = plt.subplots()
losses = np.array(losses)
plt.plot(losses.T[0], label='Discriminator', alpha=0.5)
plt.plot(losses.T[1], label='Generator', alpha=0.5)
plt.title("Training Losses")
plt.legend()

Plot the eigenlosses

In [ ]:
plt.plot(eigen_losses, label='Eigenloss', color='green')
plt.title("Eigenloss Over Time")
plt.xlabel("Epoch")
plt.ylabel("Eigenloss")
plt.legend()
plt.show()

### Generate a new set of faces

In [ ]:
# make a new set of images

generator.eval()
latent_vector = torch.randn(16, latent_dim, 1, 1).float().to(device)
generated_images = generator(latent_vector)
display(generated_images)

Use the checkpoint

In [ ]:
best_generator = Generator(latent_dim).to(device)
best_generator.load_state_dict(torch.load('generator.pth'))
best_generator.eval()
latent_vector = torch.randn(16, latent_dim, 1, 1).float().to(device)
generated_images = best_generator(latent_vector)
display(generated_images)


### Question: What do you notice about your generated samples and how might you improve this model?
When you answer this question, consider the following factors:
* The dataset is biased; it is made of "celebrity" faces that are mostly white
* Model size; larger models have the opportunity to learn more features in a data feature space
* Optimization strategy; optimizers and number of epochs affect your final result
* Loss functions

## Answer:

### Data qualities

- Bias: This is pretty much an obvious issue, stemming from the biased datbase being used. A wider representative sample of human faces would make for a more representative generative model -- not just in terms of race, but also in terms of age and other characteristics, such as facial expressions. A wider datbase of "faces in the wild" may produce a more representative sample.

- More data. Parallel with this notebook I had previous run the same architecture but with the larger CelebA dataset. There with a 128 batch size each epoch trained on 1583 batches. This resulted in better quality generations sooner. More data will give the model more diversity here as well.

- Less cropping. In the other experimentation mentined above, without the radical cropping, the generated images became more indivisualized. So data quality may be significant: cropping, resolution, etc.

For example, compare generations with the same architecture and amount of training (15 epochs), here:

<img src='https://github.com/karencfisher/GAN-experiments/blob/main/dlnd_face_generation/assets/generations1.png?raw=1' width=60% />

And in my other experimentation (https://github.com/karencfisher/GAN-experiments/blob/main/celeba.ipynb), with more data and less preprocessing:

<img src='https://github.com/karencfisher/GAN-experiments/blob/main/dlnd_face_generation/assets/generations2.png?raw=1' width=60% />

The latter to me looks more like people you might expect to meet or at least see in passing. Especially the first, second, and last, examples in the second row, which are remarkably realistic.

## Model Complexity

- I deepend the model by an additional ConvBlock to the discriminator and DeconvBlock to the generator to capture more nuances from the images. The resulting generated images began to have more dimension as well as details.

- Longer training. I did one run of 30 epochs, and then a second run with the same weigths from the first. With the added layer plus additional training time the generator began to integrate nuances such as glasses and facial hair.

## A New Research Idea

I have a concept that I am exploring: an additional loss value for the generator using cosine distance between generated images and a set of eigenfaces generated from a sample of the training data. That loss may be used for the purpose of early stopping if the images degrade for a reasonable level of patience, or to save the weights for the generator whenever the loss is minimum.

### Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_face_generation.ipynb".